In [1]:
from DeepER import init_embeddings_index, init_embeddings_model, init_DeepER_model, train_model_ER, replace_last_layer, model_statistics
from experimental_similarity import mono_vector, cosine_similarity_vector, distance_similarity_vector
from csv2dataset import splitting_dataSet, csv_2_datasetALTERNATE, csvTable2datasetRANDOM,parsing_anhai_data
from generate_similarity_vector import generate_similarity_vector
from data_reg import sim_hamming
from keras.models import load_model
from random import shuffle
import utilist as uls

Using TensorFlow backend.


In [2]:
# Caricamento strutture dati e modelli ausiliari.
embeddings_index = init_embeddings_index('embeddings/glove.6B.100d.txt')
emb_dim = len(embeddings_index['cat']) # :3
embeddings_model, tokenizer = init_embeddings_model(embeddings_index)

* Costruzione indice degli embeddings.....Fatto. 400000 embeddings totali.
* Creazione del modello per il calcolo degli embeddings....
* Inizializzo il tokenizzatore.....Fatto: 400000 parole totali.
* Preparazione della matrice di embedding.....Fatto. Dimensioni matrice embeddings: (400001, 100)

°°° EMBEDDING MODEL °°°
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            (None, None)         0                                            
_____________________________________________________________________________________

In [3]:
LOAD_FROM_DISK_DATASET = True
DATASET_NAME ='walmart_amazon'

In [4]:
# Caricamento dati e split iniziale.
if LOAD_FROM_DISK_DATASET:
    
    # Carica dataset salvato su disco.
    data = uls.load_list(f'dataset_{DATASET_NAME}')
    match_number=sum(map(lambda x : x[3] == 1, data))
    print("match_number: " + str(match_number))
    print("len all dataset: "+ str(len(data)))

else:
    
    GROUND_TRUTH_FILE = 'matches_fodors_zagats.csv'# Esempio: 'matches_walmart_amazon.csv'
    # Necessario inserire le tabelle nell'ordine corrispondente alle coppie della ground truth.
    TABLE1_FILE = 'fodors.csv'# Esempio: 'walmart.csv'
    TABLE2_FILE = 'zagats.csv'# Esempio: 'amazon.csv'

    # Coppie di attributi considerati allineati.
    att_indexes = [(1, 1), (2, 2), (3, 3), (4, 4),(5, 5), (6, 6)]# Esempio: [(5, 9), (4, 5), (3, 3), (14, 4), (6, 11)]


    # Crea il dataset.
    data = csv_2_datasetALTERNATE(GROUND_TRUTH_FILE, TABLE1_FILE, TABLE2_FILE, att_indexes)
    #per i dataset di Anhai
    #data=parsing_anhai_data(GROUND_TRUTH_FILE, TABLE1_FILE, TABLE2_FILE, att_indexes)
    
    # Salva dataset su disco.
    uls.save_list(data, f'dataset_{DATASET_NAME}')

    
# Dataset per DeepER classico: [(tupla1, tupla2, label), ...].
deeper_data = list(map(lambda q: (q[0], q[1], q[3]), data))


# Taglia attributi se troppo lunghi
# Alcuni dataset hanno attributi con descrizioni molto lunghe.
# Questo filtro limita il numero di caratteri di un attributo a 1000.
def shrink_data(data):
    
    def cut_string(s):
        if len(s) >= 1000:
            return s[:1000]
        else:
            return s
    
    temp = []
    for t1, t2, lb in data:
        t1 = list(map(cut_string, t1))
        t2 = list(map(cut_string, t2))
        temp.append((t1, t2, lb))
        
    return temp

deeper_data = shrink_data(deeper_data)


# Split in training set e test set.
def split_training_test(data, SPLIT_FACTOR = 0.8):     
    bound = int(len(data) * SPLIT_FACTOR)
    train = data[:bound]
    test = data[bound:]
    
    return train, test


# Tutti i successivi addestramenti partiranno dal 100% di deeper_train (80% di tutti i dati).
# Le tuple in deeper_test non verranno mai usate per addestrare ma solo per testare i modelli.
deeper_train, deeper_test = split_training_test(deeper_data)

match_number: 1154
len all dataset: 2308


## Utilities functions

In [5]:
from keras.models import Model
from keras import backend as K
from keras.layers import Input

def get_layer_output(model,layer_name,data):
    intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(data)
    return intermediate_output

In [6]:
def get_layer_output_grad(model, inputs, outputs, layer_name):
    """ Gets gradient a layer output for given inputs and outputs"""
    grads = model.optimizer.get_gradients(model.total_loss, model.get_layer(layer_name).output)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad

In [7]:
def build_submodel_fromModel(model,layer_idx,input_shape):
    layer_input = Input(shape= input_shape) # a new input tensor to be able to feed the desired layer
    # create the new nodes for each layer in the path
    x = layer_input
    for layer in walmart_amazon_model.layers[layer_idx:]:
        x = layer(x)
    # create the model
    new_model = Model(layer_input, x)
    return new_model

## Load model and extract layer input

In [8]:
def isNegative(sample):
    return sample[2]==0
train_negatives = list(filter(isNegative,deeper_train))

In [9]:
len(deeper_train),len(train_negatives)

(1846, 923)

In [10]:
from DeepER import data2Inputs
table1, table2, labels = data2Inputs(deeper_train, tokenizer, categorical=False)
embeddings = embeddings_model.predict([table1,table2])
table1_neg,table2_neg, labels = data2Inputs(train_negatives,tokenizer,categorical=False)
negEmbeddings = embeddings_model.predict([table1_neg,table2_neg])

* Preparazione input......Fatto. 1846 tuple totali, esempio label: 1 -> 1, Table1 shape: (1846, 33), Table2 shape: (1846, 13)

* Preparazione input......Fatto. 923 tuple totali, esempio label: 0 -> 0, Table1 shape: (923, 33), Table2 shape: (923, 12)


In [11]:
walmart_amazon_model = load_model('models/DeepER_best_model_100_walmart_amazon.h5')
walmart_amazon_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Embeddings_seq_a (InputLayer)   (None, None, 100)    0                                            
__________________________________________________________________________________________________
Embeddings_seq_b (InputLayer)   (None, None, 100)    0                                            
__________________________________________________________________________________________________
Composition (Bidirectional)     (None, 300)          301200      Embeddings_seq_a[0][0]           
                                                                 Embeddings_seq_b[0][0]           
__________________________________________________________________________________________________
Similarity (Lambda)             (None, 300)          0           Composition[0][0]          

In [12]:
negPredictions = walmart_amazon_model.predict(negEmbeddings)
similarity_output_neg = get_layer_output(walmart_amazon_model,'Similarity',negEmbeddings)

In [13]:
selected_weights = []
for layer in walmart_amazon_model.layers[-3:]:
    selected_weights.append(layer.get_weights())

## Some tests

In [ ]:
dense1_output = get_layer_output(walmart_amazon_model,'Dense1',embeddings)
dense2_output = get_layer_output(walmart_amazon_model,'Dense2',embeddings)

In [ ]:
classication_output = softmax(getActivation(classification_weights[1],classification_weights[0],dense2_output))

In [ ]:
predictions[0:4]

In [ ]:
classification_out_real = get_layer_output(walmart_amazon_model,'Classification',embeddings)
classification_out_real[1]

In [44]:
import tensorflow as tf 
import numpy as np
outputTensor = walmart_amazon_model.output
trainableWeights = walmart_amazon_model.trainable_weights
gradients = K.gradients(outputTensor,trainableWeights)
l_exampleInput = embeddings[0][0:8]
r_exampleInput = embeddings[1][0:8]

with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    evaluated_gradients = s.run(gradients,feed_dict={walmart_amazon_model.input[0]:l_exampleInput,
                                                    walmart_amazon_model.input[1]:r_exampleInput})

In [108]:
similarity_output = walmart_amazon_model.layers[-4].output
model_output = walmart_amazon_model.output
custom_loss = K.max(model_output,axis=1)
gradients = K.gradients(custom_loss,walmart_amazon_model.get_layer('Dense1').input)
with tf.Session() as s:
    s.run(tf.global_variables_initializer())
    evaluated_gradients = s.run(gradients,feed_dict={walmart_amazon_model.input[0]:l_exampleInput,
                                                    walmart_amazon_model.input[1]:r_exampleInput})

In [109]:
for grad in evaluated_gradients[0]:
    print (np.linalg.norm(grad))

0.30017543
0.29355785
0.27970394
0.26048666
0.28302205
0.2617454
0.25232735
0.23955864


## Compute gradients

In [ ]:
import tensorflow as tf 
from tensorflow.nn import relu,softmax
from tensorflow.linalg import matmul

In [ ]:
def createComputationGraphForGradient(weight_list,activation_f,input_batch):
    input_t = tf.constant(input_batch)
    layer_output = [input_t]
    for i in range(len(weight_list)):
        current_layer_weight = tf.constant(weight_list[i][0])
        current_layer_bias = tf.constant(weight_list[i][1])
        current_output = activation_f[i](matmul(layer_output[i],current_layer_weight)+current_layer_bias)
        layer_output.append(current_output)
    final_output_t = layer_output[-1]
    return (input_t,final_output_t)

In [ ]:
config=tf.ConfigProto(log_device_placement=True)
session = tf.Session(config=config)

In [ ]:
activations = [relu,relu,softmax]
initial_gradients = np.zeros((64,2)).astype(np.float32)
initial_gradients[:,1]=1
initial_gradients_t = tf.constant(initial_gradients)
x,y = createComputationGraphForGradient(selected_weights,activations,similarity_output_neg[0:64])

In [ ]:
y = tf.constant(initial_gradients)
gradients = tf.gradients(y,x)

In [ ]:
gradients0_15 = session.run(gradients)

In [ ]:
for grad in gradients0_15[0]:
    print(np.linalg.norm(grad))

In [ ]:
session.close()

## Define Ri algorithm

In [ ]:
import numpy as np
from numpy.linalg import norm

def find_smallest_variation_to_change(weight_list,activations,input_batch,sample_idx,classifier_length,class_to_reach,
                                     tf_session):
    xi = input_batch[sample_idx]
    input_batch_copy = input_batch.copy()
    input_t,output_t = createComputationGraphForGradient(weight_list,activations,input_batch_copy)
    current_probabilities = tf_session.run(output_t)[sample_idx]
    sum_ri = np.zeros(classifier_length)
    iterations = 0
    while(round(current_probabilities[1])!=class_to_reach and iterations<50):
        print("current iteration is {}".format(str(iterations)))
        input_t,output_t = createComputationGraphForGradient(weight_list,activations,input_batch_copy)
        current_probabilities = tf_session.run(output_t)[sample_idx]
        if class_to_reach == 1:
            fx = current_probabilities[0]
        else:
            fx = current_probabilities[1]
        current_gradient = tf_session.run(tf.gradients(output_t,input_t))[0][sample_idx]
        current_norm = norm(current_gradient)
        if current_norm==0:
            sum_ri = np.zeros(classifier_length)
            print("Gradient is null")
            break
        ri = (fx/(current_norm**2)) * (-current_gradient)
        xi = xi+ri
        input_batch_copy[sample_idx] = xi
        
        sum_ri += ri
        iterations+=1
    if iterations>=50:
        sum_ri = np.zeros(classifier_length)
        print("can't converge ")
        
    return sum_ri

In [ ]:
input_batch = similarity_output_neg
sample_idx = 0
classifier_length = 300
class_to_reach =0
tf_session = tf.Session()

In [ ]:
from tqdm import tqdm
ris = []
for i in tqdm(range(len(input_batch))):
    ri = find_smallest_variation_to_change(selected_weights,activations,input_batch,i,classifier_length=300,class_to_reach=1,
                                      tf_session=tf_session)
    ris.append(ri)
tf_session.close()